In [70]:
%matplotlib widget

In [71]:
import torch
import pyro
from pyro.infer import mcmc

In [72]:
from collections import OrderedDict

In [76]:
def half_life_one_isotope(obs):
    daily_body_waste = pyro.sample('daily_body_waste', pyro.distributions.Exponential(1.))
    half_life = pyro.sample('half_life', pyro.distributions.Exponential(1.))
    
    likelihoods = []
    l0 = obs['l0']
    t0 = obs['t0']
    for fi, fo in enumerate(obs['followup']):
        t_ = fo['time']-t0
        level_ = fo['level']
    
        mu_ = ((1./2.) ** (t_ / half_life)) * l0 - daily_body_waste * t_
        lik_ = pyro.sample('obs_{}'.format(fi), pyro.distributions.Normal(mu_, 1.), obs=torch.ones(1) * level_) 
        likelihoods.append(lik_)
    
    return likelihoods

In [77]:
timestamps = [43618.65625,
            43618.70833,
            43618.74653,
            43618.82292,
            43618.89236,
            43618.95486,
            43619.38889,
            43619.4375]
measurements = [
    6600,
    6300,
    6600,
    5800,
    5300,
    4900,
    850,
    1050,
]

In [78]:
obs = OrderedDict()

In [79]:
obs['t0'] = timestamps[0]
obs['l0'] = measurements[0]

In [80]:
obs['followup'] = []
for tt, mm in zip(timestamps, measurements):
    obs['followup'].append(OrderedDict({'time': tt, 'level': mm}))

In [81]:
nuts_kernel = pyro.infer.mcmc.NUTS(half_life_one_isotope, adapt_step_size=True, step_size=0.1)
mcmc_run = pyro.infer.mcmc.MCMC(nuts_kernel, num_samples=200, warmup_steps=50).run(obs)
score_marginal = pyro.infer.EmpiricalMarginal(mcmc_run, sites=["half_life", "daily_body_waste"])

Sample: 100%|██████████| 250/250 [06:35<00:00,  1.42s/it, step size=1.93e-04, acc. rate=0.992]


In [91]:
print('estimated half life (hours)', 24 * score_marginal.mean[0].item(), '±', 24 * score_marginal.stddev[0].item())

estimated half life (hours) 10.3615243434906 ± 0.0026238035643473268


In [93]:
print('estimated cpm body exit per day (not accounted by radioactive decay)', score_marginal.mean[1].item(), '±', score_marginal.stddev[1].item())

estimated cpm body exit per day (not accounted by radioactive decay) 1.1821047067642212 ± 0.5966048836708069
